# Image rotation using OpenCV 
### [(Example using phase contrast microsocopy data)](https://www.kaggle.com/c/sartorius-cell-instance-segmentation)
## Author: [Dr. Rahul Remanan](https://www.linkedin.com/in/rahulremanan/)
## CEO, [Moad Computer](https://moad.computer)
In this notebook, the input images using the phase contrast microscopy images and their corresponding mask labels, from the [Sartorius cell instance segmentation challenge data-set in Kaggle](https://www.kaggle.com/c/sartorius-cell-instance-segmentation), are rotated to a specific angle (in degrees) using [OpenCV](https://opencv.org/).

# Import libraries

In [ ]:
import cv2, math, random, numpy as np, pandas as pd, \
       tensorflow as tf, matplotlib.pyplot as plt

# Initialize variables

In [ ]:
ROOT_DIR = '/kaggle/input'
DATA_DIR = f'{ROOT_DIR}/sartorius-cell-instance-segmentation'
TRAIN_CSV = f'{DATA_DIR}/train.csv'

INPUT_HEIGHT              = 520
INPUT_WIDTH               = 704
IMAGE_HEIGHT              = 512
IMAGE_WIDTH               = 512
INPUT_CHANNELS            = 3
IMAGE_CHANNELS            = 3
INPUT_SIZE                = [INPUT_HEIGHT, INPUT_WIDTH]
IMAGE_SIZE                = [IMAGE_HEIGHT, IMAGE_WIDTH]

# Read images and mask files

In [ ]:
df = pd.read_csv(TRAIN_CSV)
display(df.cell_type.unique())
display(df.cell_type.hist())

In [ ]:
#ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, mask, color=1):
  '''
  mask_rle: run-length as string formated (start length)
  shape: (height, width, channels) of array to return 
  color: color for the mask
  Returns numpy array (mask)
  '''
  s = mask_rle.split()
  starts = list(map(lambda x: int(x) - 1, s[0::2]))
  lengths = list(map(int, s[1::2]))
  ends = [x + y for x, y in zip(starts, lengths)]
  img = mask.reshape((mask.shape[0] * mask.shape[1]))
  for start, end in zip(starts, ends):
    img[start : end] = color
  return img.reshape(mask.shape)

In [ ]:
def read_mask(imgID, img_size):
  mask = np.zeros(img_size, dtype=np.uint8)
  for i in range(len(df[df.id == imgID])):
    mask += rle_decode(df[df.id == imgID].annotation.iloc[i], mask)
  mask = np.clip(mask, 0,1)  
  return mask

In [ ]:
def read_png(filename):
  img   = tf.io.read_file(filename)
  img   = tf.image.decode_png(img, channels=INPUT_CHANNELS)
  img   = tf.expand_dims(img, -1)
  img   = tf.cast(img, tf.float32)
  img   = img / 255.
  img   = tf.reshape(img, [*INPUT_SIZE, INPUT_CHANNELS]) 
  img   = tf.image.resize(img, IMAGE_SIZE)
  img   = tf.reshape(img, [*IMAGE_SIZE, IMAGE_CHANNELS])
  return img

# Rotate an image using OpenCV

In [ ]:
#ref: https://stackoverflow.com/a/69324068
def rotate(img, deg, rot_point=None):
  warp_mat = np.zeros((2,3))
  rad = math.radians(deg)  
  cos, sin = np.cos(rad), np.sin(rad)
  warp_mat[:2,:2] = [[cos, -sin],[sin, cos]]
  if rot_point is None:
    rot_point = np.array(img.shape[:2][::-1])/2
  warp_mat[:2,2] = rot_point - np.matmul(warp_mat[:2,:2], rot_point)
  return cv2.warpAffine(img, warp_mat, img.shape[:2][::-1])

# Display image and its corresponding mask

In [ ]:
def display_masked_image(img, mask):  
  img   = img[...,0]
  dim   = img.shape
  img   = cv2.resize(img, (dim[0], dim[1]))
  mask  = cv2.resize(mask, (dim[0], dim[1]))
    
  plt.figure(figsize=(15, 15))
  plt.subplot(1, 3, 1)
  plt.imshow(img,cmap='gray')
  plt.title('Image') 
  plt.axis('off')
   
  plt.subplot(1, 3, 2)
  plt.imshow(mask,cmap='inferno')
  plt.title('Mask')
  plt.axis('off')
 
  plt.subplot(1, 3, 3)
  plt.imshow(img)
  plt.imshow(mask, alpha=0.4, cmap='inferno')
  plt.title('Image + Mask')
  plt.axis('off')
  plt.tight_layout()
  plt.show()

# Rotate an example image

In [ ]:
split = 'train'
img_files = tf.io.gfile.glob(f'{DATA_DIR}/{split}/*.png')
img_file = random.SystemRandom().choice(img_files)
img_id = img_file.split('/')[-1].replace('.png', '')
print(f'Reading image file id: {img_id} ...')
img = read_png(img_file).numpy()
mask = read_mask(img_id, tuple(INPUT_SIZE))
mask = cv2.resize(mask, img.shape[:2][::-1])

In [ ]:
r_deg = 90
r_img = rotate(img, r_deg)
r_mask = rotate(mask, r_deg)

# Display the input image, its mask and the rotated output

In [ ]:
print('='*10,'Original image and mask','='*10)
display(display_masked_image(img, mask))
print('='*10,'Rotated image and mask','='*10)
display(display_masked_image(r_img, r_mask))

### References:
1. [Image rotation with-respect-to a specific point on an image](https://stackoverflow.com/a/69324068)